In [ ]:
! pip install scikit-surprise

     |████████████████████████████████| 11.8MB 514kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617642 sha256=722c794c8cd62b8e52b144a696599d52faefef2eb2655105e59d5d8725682e1e
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from math import sqrt
import time
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split as train_test_split_sk
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


## 1. Data Preparation

In [ ]:
##Load Movie Title data
df_titles = pd.read_csv('/content/drive/My Drive/Web Mining/Org. Dataset for Netflix/movie_titles.csv', encoding = "ISO-8859-1", delimiter=',', names=['id', 'release_year', 'title'])
print(df_titles.shape)

(17770, 3)


In [ ]:
df_merged = pd.read_csv("/content/drive/My Drive/Web Mining/NEW_DATA/final_movies_metadata_linked.csv", delimiter=';')
df_merged.drop(['Unnamed: 0'], axis=1, inplace=True)
df_merged.head()

,netflix_id,title,release_year,metadata_id,imdb_id,original_language,overview,tagline,metadata_genres,netflix_genres
0,402,Pan Tadeusz,1999,4966,tt0170351,pl,A grand and patriotic tale of Poland's struggl...,NaN,"[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...",Drama|History|Romance|War
1,17617,Ginger,1971,89072,tt0067133,en,The police suspect that a drug and forced-pros...,"Every man wanted her, no man could tame her! T...","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",Action|Crime|Drama
2,11805,The Big Bounce,2004,12634,tt0315824,en,A small-time con artist and a Hawaiian real es...,It's all in who you trust,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",Comedy|Crime
3,459,Basquiat,1996,549,tt0115632,en,Director Julian Schnabel illustrates the portr...,"In 1981, A Nineteen-Year-Old Unknown Graffiti ...","[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",Biography|Drama
4,4607,Melody Time,1948,13757,tt0040580,en,Melody Time is an animated feature produced by...,NaN,"[{'id': 10402, 'name': 'Music'}, {'id': 10751,...",Animation|Comedy|Family|Musical


In [ ]:
df_merged.rename(columns={'metadata_id' : 'movie_id', 'netflix_genres' : 'genre'}, inplace=True)

df_merged.head()

,netflix_id,title,release_year,movie_id,imdb_id,original_language,overview,tagline,metadata_genres,genre
0,402,Pan Tadeusz,1999,4966,tt0170351,pl,A grand and patriotic tale of Poland's struggl...,NaN,"[{'id': 10752, 'name': 'War'}, {'id': 18, 'nam...",Drama|History|Romance|War
1,17617,Ginger,1971,89072,tt0067133,en,The police suspect that a drug and forced-pros...,"Every man wanted her, no man could tame her! T...","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",Action|Crime|Drama
2,11805,The Big Bounce,2004,12634,tt0315824,en,A small-time con artist and a Hawaiian real es...,It's all in who you trust,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",Comedy|Crime
3,459,Basquiat,1996,549,tt0115632,en,Director Julian Schnabel illustrates the portr...,"In 1981, A Nineteen-Year-Old Unknown Graffiti ...","[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",Biography|Drama
4,4607,Melody Time,1948,13757,tt0040580,en,Melody Time is an animated feature produced by...,NaN,"[{'id': 10402, 'name': 'Music'}, {'id': 10751,...",Animation|Comedy|Family|Musical


In [ ]:
df_metadata=df_merged[['movie_id','title','release_year','overview','genre']]
df_metadata.fillna("", inplace=True)
df_metadata.head()


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,movie_id,title,release_year,overview,genre
0,4966,Pan Tadeusz,1999,A grand and patriotic tale of Poland's struggl...,Drama|History|Romance|War
1,89072,Ginger,1971,The police suspect that a drug and forced-pros...,Action|Crime|Drama
2,12634,The Big Bounce,2004,A small-time con artist and a Hawaiian real es...,Comedy|Crime
3,549,Basquiat,1996,Director Julian Schnabel illustrates the portr...,Biography|Drama
4,13757,Melody Time,1948,Melody Time is an animated feature produced by...,Animation|Comedy|Family|Musical


In [ ]:
Mapping_file = dict(zip(df_metadata.title.tolist(), df_metadata.movie_id.tolist()))

In [ ]:
df_ratings = pd.read_csv("/content/drive/My Drive/Web Mining/NEW_DATA/Truncated/truncated_ratings.csv", delimiter=';')
df_ratings.drop(['Unnamed: 0'], axis=1, inplace=True)
df_ratings.shape

(4265493, 4)

In [ ]:
#Create dictionary of movie_id to title so that we can use it later to determine title when recommending items
dict_id_title = df_merged.set_index("netflix_id")["title"].to_dict()

In [ ]:
##Create Movies dataframe to explore movie's avg rating and number of ratings received by user
df_titles = pd.DataFrame(df_ratings.groupby('movie_id')['rating'].mean())
df_titles['count'] = df_ratings.groupby('movie_id')['rating'].count()
df_titles["title"] = pd.Series(dict_id_title)
df_titles.head().sort_values('count')

,rating,count,title
movie_id,,,
12,3.423810,420,My Favorite Brunette
5,3.635535,439,WWE: The Rise & Fall of ECW
3,3.556656,909,Character
17,2.872961,2330,7 Seconds
8,3.088164,4968,What the #$*! Do We (K)now!?


In [ ]:
df_users = pd.DataFrame(df_ratings.groupby('user_id')['rating'].mean())
df_users['count'] = df_ratings.groupby('user_id')['rating'].count()
df_users.sort_values(['count'])

,rating,count
user_id,,
2481919,3.419355,31
724599,3.419355,31
1186037,3.250000,32
1416038,2.781250,32
184751,3.968750,32
...,...,...
1664010,4.528994,845
2118461,4.152403,853
2439493,1.284553,861


In [ ]:
df_users_f = df_users.drop(df_users[df_users['count'] <150].index)
df_titles_f = df_titles.drop(df_titles[df_titles['count'] < 50].index)

In [ ]:
print(df_titles.shape)
print(df_titles_f.shape)
print(df_users.shape)
print(df_users_f.shape)

print('Preserved rate of the movies : {0:.2f} %'.format((df_titles_f.shape[0]/df_titles.shape[0]) * 100))
print('Preserved rate of the users : {0:.2f} %'.format((df_users_f.shape[0]/df_users.shape[0]) * 100))

(909, 3)
(900, 3)
(57357, 2)
(2225, 2)
Preserved rate of the movies : 99.01 %
Preserved rate of the users : 3.88 %


In [ ]:
list_users_f = list(df_users_f.index.values)
list_titles_f = list(df_titles_f.index.values)

print(len(list_users_f))
print(len(list_titles_f))

2225
900


In [ ]:
df_ratings_f = df_ratings[df_ratings['user_id'].isin(list_users_f)]
print(df_ratings.shape)
print(df_ratings_f.shape)

(4265493, 4)
(431752, 4)


# Find Best Model by Training on Subset of Netflix Data


###Hyperparameter Tuning

In [ ]:
# Cross Validation
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader
from surprise import KNNWithMeans,KNNWithZScore

reader = Reader(rating_scale=(0, 5))
trainset, testset = train_test_split(df_ratings_f[['user_id','movie_id','rating']], test_size=0.25, random_state=0)
df_test =pd.DataFrame(testset,columns= ['user_id','movie_id','rating'])
print ("Test Data Size %i" %len(df_test))
trainset = Dataset.load_from_df(trainset, reader)
data = Dataset.load_from_df(df_ratings_f[['user_id','movie_id','rating']], reader)
testset = testset.values.tolist()

# Define the models and their hyperparameters 
sim_option_user_cosine = {'name': 'cosine',
              'user_based': False   # compute similarities between users
               }
sim_option_user_pearson = {'name': 'pearson',
              'user_based': False  # compute similarities between users
}

benchmark = []
i =0
alg_list = ['KNNWithMeans(k=30, sim=cosine, item-based)',
            'KNNWithMeans(k=40, sim=cosine, item-based)',
            'KNNWithMeans(k=50, sim=cosine, item-based)',
            'KNNWithMeans(k=30, sim=pearson, item-based)',
            'KNNWithMeans(k=40, sim=pearson, item-based)',
            'KNNWithMeans(k=50, sim=pearson, item-based)']

for algorithm in [KNNWithMeans(k=30,sim_options=sim_option_user_cosine),
                 KNNWithMeans(k=40,sim_options=sim_option_user_cosine),
                 KNNWithMeans(k=50,sim_options=sim_option_user_cosine),
                 KNNWithMeans(k=30,sim_options=sim_option_user_pearson),
                 KNNWithMeans(k=40,sim_options=sim_option_user_pearson),
                 KNNWithMeans(k=50,sim_options=sim_option_user_pearson)]:


    # Perform cross validation
    results = cross_validate(algorithm, trainset, measures=['RMSE','MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([alg_list[i]], index=['Algorithm']))
    benchmark.append(tmp)
    i+=1
    

Test Data Size 107938
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarit

In [ ]:
  pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
"KNNWithMeans(k=30, sim=cosine, item-based)",0.863156,0.674094,2.225934,11.656091
"KNNWithMeans(k=40, sim=cosine, item-based)",0.864992,0.676136,2.143270,12.859512
"KNNWithMeans(k=50, sim=cosine, item-based)",0.867302,0.678379,2.160025,13.787127
"KNNWithMeans(k=40, sim=pearson, item-based)",0.869570,0.678852,3.069048,13.228512
"KNNWithMeans(k=50, sim=pearson, item-based)",0.869728,0.679470,3.058285,14.255937
"KNNWithMeans(k=30, sim=pearson, item-based)",0.871423,0.679999,2.953346,11.856589


In [ ]:
from surprise import KNNWithMeans

trainset = trainset.build_full_trainset()

algorithm = KNNWithMeans(k=30,sim_options=sim_option_user_cosine)

model_KnnwithMeans = algorithm.fit(trainset)

predictions = model_KnnwithMeans.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
from collections import Counter
df_pred = pd.DataFrame(predictions,columns = ['user_id','movie_id','rating','pred_rating','details'])
df_pred.head(10)

,user_id,movie_id,rating,pred_rating,details
0,2062350,1058,5,3.403004,"{'actual_k': 30, 'was_impossible': False}"
1,1118103,1103,4,3.297996,"{'actual_k': 30, 'was_impossible': False}"
2,518137,166,4,3.754322,"{'actual_k': 30, 'was_impossible': False}"
3,46650,1708,3,3.078756,"{'actual_k': 30, 'was_impossible': False}"
4,2495584,1682,3,3.315601,"{'actual_k': 30, 'was_impossible': False}"
5,1728944,947,3,3.641561,"{'actual_k': 30, 'was_impossible': False}"
6,2576843,401,2,3.338031,"{'actual_k': 30, 'was_impossible': False}"
7,1090319,285,3,2.850659,"{'actual_k': 30, 'was_impossible': False}"
8,232877,187,2,2.515693,"{'actual_k': 30, 'was_impossible': False}"
9,2543231,165,3,3.822791,"{'actual_k': 30, 'was_impossible': False}"


In [ ]:
# RecSys
def pred_user_rating(userid, num_recommend):
    if userid in df_ratings_f.user_id.unique():
        ui_list = df_ratings_f[df_ratings_f.user_id == user_id].movie_id.tolist()
        d = {k: v for k,v in Mapping_file.items() if not v in ui_list}        
        predictedL = []
        for i, j in d.items():     
            predicted = algorithm.predict(user_id, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns = ['movies', 'ratings'])
        pdf.sort_values('ratings', ascending=False, inplace=True)  
        pdf.set_index('movies', inplace=True)    
        return pdf.head(num_recommend)        
    else:
        print("User Id does not exist in the list!")
        return None

In [ ]:
user_id = int(input("Enter the user id to whom you want to recommend : "))
num_recommend = int(input("Enter the number of movies you want to recommend : "))
pred_user_rating(user_id, num_recommend)

Enter the user id to whom you want to recommend : 2062350
Enter the number of movies you want to recommend : 10


,ratings
movies,
Arlington Road,4.576494
The Cincinnati Kid,4.455880
Garden State,4.002686
The Godfather: Part II,4.002441
Closely Watched Trains,3.994001
"Monsters, Inc.",3.979533
Batman,3.968087
Umberto D.,3.960935
The Man with the Golden Gun,3.955001


In [ ]:
print("Get recommended movies metadata")
rec_movies = pred_user_rating(user_id, num_recommend)
rec_moves_md = pd.merge(rec_movies, df_metadata , how='left', left_on='movies', right_on='title')[['title','release_year','overview','genre']]
display(rec_moves_md.head(10))


print("")
print("Movies that User rated")
user_movies = df_ratings_f[df_ratings_f.user_id==2062350].merge(df_metadata).sort_values('rating', ascending=False)

user_movies.head(10)

Get recommended movies metadata


,title,release_year,overview,genre
0,Arlington Road,1999,Threats from sinister foreign nationals aren't...,Crime|Drama|Thriller
1,The Cincinnati Kid,1965,An up-and-coming poker player tries to prove h...,Drama
2,Garden State,2004,Andrew returns to his hometown for the funeral...,Comedy|Drama|Romance
3,The Godfather: Part II,1974,In the continuing saga of the Corleone crime f...,Crime|Drama
4,Closely Watched Trains,1966,In a small town still occupied by the Germans ...,Comedy|Drama|Romance|War
5,"Monsters, Inc.",2001,"James Sullivan and Mike Wazowski are monsters,...",Animation|Adventure|Comedy|Family|Fantasy
6,Batman,1989,The Dark Knight of Gotham City begins his war ...,Action|Adventure
7,Umberto D.,1952,"Umberto Domenico Ferrari, an elderly and retir...",Drama
8,The Man with the Golden Gun,1974,A golden bullet has 007 engraved on it as it s...,Action|Adventure|Thriller
9,Requiem for a Dream,2000,The hopes and dreams of four ambitious people ...,Drama



Movies that User rated


,movie_id,user_id,rating,rating_date,title,release_year,overview,genre
0,28,2062350,5,2003-06-16,Apocalypse Now,1979,"At the height of the Vietnam war, Captain Benj...",Drama|Mystery|War
65,862,2062350,5,2005-01-04,Toy Story,1995,"Led by Woody, Andy's toys live happily in his ...",Animation|Adventure|Comedy|Family|Fantasy
32,468,2062350,5,2004-12-21,My Own Private Idaho,1991,"In this loose adaptation of Shakespeare's ""Hen...",Drama
37,535,2062350,5,2003-06-15,Flashdance,1983,The popular 1980’s dance movie that depicts th...,Drama|Music|Romance
42,571,2062350,5,2003-06-17,The Birds,1963,Chic socialite Melanie Daniels enjoys a passin...,Drama|Horror|Mystery|Romance
46,615,2062350,5,2003-06-16,The Passion of the Christ,2004,"""The Passion of the Christ"" is a film about th...",Drama
48,629,2062350,5,2004-12-13,The Usual Suspects,1995,"Held in an L.A. interrogation room, Verbal Kin...",Crime|Mystery|Thriller
50,640,2062350,5,2003-06-23,Catch Me If You Can,2002,"A true story about Frank Abagnale Jr. who, bef...",Biography|Crime|Drama
55,696,2062350,5,2003-06-16,Manhattan,1979,The life of a divorced television writer datin...,Comedy|Drama|Romance
56,705,2062350,5,2003-06-17,All About Eve,1950,From the moment she glimpses her idol at the s...,Drama


# Split Full Ratings data into Training, Validation and Test Set

---



In [ ]:
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader, accuracy
from surprise import KNNWithMeans,KNNWithZScore


# Instantiate a reader with the specified rating scale
# and read in the rating data
reader = Reader(rating_scale=(0, 5))

#rawTrain = pd.read_csv("/content/drive/My Drive/Web Mining/NEW_DATA/train_data_ratings.csv",delimiter=';')
#rawTest = pd.read_csv("/content/drive/My Drive/Web Mining/NEW_DATA/test_data_ratings.csv",delimiter=';')
rawTrain = pd.read_csv("/content/drive/My Drive/Web Mining Project/NEW_DATA/train_data_ratings.csv",delimiter=';')
rawTest = pd.read_csv("/content/drive/My Drive/Web Mining Project/NEW_DATA/test_data_ratings.csv",delimiter=';')
rawTrain.drop(['Unnamed: 0'],1,inplace=True)
rawTest.drop(['Unnamed: 0'],1,inplace=True)

print(rawTrain.info())
print(rawTest.info())

In [ ]:
#into surprise:
rawTrain = Dataset.load_from_df(rawTrain,reader) #used to build validation set as well
rawTest = Dataset.load_from_df(rawTest,reader)

In [ ]:
rawTrain = rawTrain.build_full_trainset()
rawTest = rawTest.build_full_trainset().build_testset()

In [ ]:
sim_option_user_cosine = {'name': 'cosine',
              'user_based': False  # compute similarities between items
}

algorithm = KNNWithMeans(k=30,sim_options=sim_option_user_cosine)

In [ ]:
model_KNNWithKmeans = algorithm.fit(rawTrain)

Computing the cosine similarity matrix...


In [ ]:
predictions = model_KNNWithKmeans.test(rawTest)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.8549


0.85489472478052

In [ ]:
test_data_df = pd.read_csv("/content/drive/My Drive/Web Mining Project/NEW_DATA/test_data_ratings.csv", delimiter=';')
test_data_df.drop(['Unnamed: 0'], axis=1, inplace=True)
test_data_df.head()

,user_id,movie_id,rating
0,1424768,1180,4
1,653392,15472,4
2,1222598,872,5
3,341108,10234,3
4,2533261,4912,5


In [ ]:
# Training the models and predicting for the users and movies in the validation data
CF_predictions = []

# For every movie in the validation data
for index, row in test_data_df.iterrows():
  CF_predictions.append(model_KNNWithKmeans.predict(row['user_id'], row['movie_id']).est)

In [ ]:
test_data_df['predicted_rating'] = CF_predictions

In [ ]:
test_data_df.to_csv('/content/drive/My Drive/Web Mining Project/NEW_DATA/Results/test_data_predictions_knn_item.csv', sep=';', encoding='utf-8', index=False)